In [ ]:
# Import required modules
import sys
from pathlib import Path

In [ ]:
project_root = str(Path.cwd().resolve().parent)
print(f"Project root: {project_root}")

In [ ]:
# Add project root to sys.path for module imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.analysis import vram_usage

# Automatically reload modules before executing code
# This is useful for development to see changes without restarting the kernel.
%load_ext autoreload
# Reload all modules imported with %aimport every time before executing the Python code typed.
%autoreload 1
%aimport src.analysis.vram_usage

In [ ]:
# Load the jobs DataFrame from DuckDB
query =  (
        """SELECT * FROM Jobs
        WHERE GPUs == 1 and  
        ('a100' IN GPUType OR 
        'h100' IN GPUType)"""
    )
efficiency_analysis = vram_usage.EfficiencyAnalysis(
    db_path='../data/slurm_data.db',
    sample_size=5000,
    random_state=42,
    query=query
)

display(efficiency_analysis.jobs_df)

In [ ]:
efficiency_jobs = efficiency_analysis.calculate_efficiency_metrics()
display(efficiency_jobs)

In [ ]:
# Run evaluate_cpu_gpu_balance
analysis_results = efficiency_analysis.evaluate_cpu_gpu_usage()
print("Efficiency summary stats:")

# Display key summary statistics
print("Total jobs:", analysis_results["total_jobs"])
print("Total GPU hours:", analysis_results["total_gpu_hours"])
print("Average VRAM efficiency:", f"{analysis_results['avg_efficiency']:.2%}")
print("Median VRAM efficiency:", f"{analysis_results['median_efficiency']:.2%}")

# Show report
print("\nReport:")
for rec in analysis_results["report"]:
    print("-", rec)
# Display efficiency patterns table
analysis_results["efficiency_patterns"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

inefficient_users = efficiency_analysis.find_inefficient_users_weighted_by_hours(
    min_jobs=5,  # Minimum number of jobs to consider a user
    efficiency_threshold=0.3  # Efficiency threshold for identifying inefficient users
)

# Display top inefficient users by job count
print("\nTop inefficient users by job count:")
display(inefficient_users.head(10))



# Plot top inefficient users by GPU hours
top_users = inefficient_users.head(10)

plt.figure(figsize=(7, 5))
sns.barplot(y=top_users["User"], x=top_users["Total_GPU_Hours"], orient="h")
plt.xlabel("GPU Hours")
plt.ylabel("User")
plt.title("Top 10 Inefficient Users by Weighted VRAM Efficiency Contribution")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

inefficient_pis = efficiency_analysis.find_inefficient_pis_weighted_by_hours(
    min_jobs=5,  # Minimum number of jobs to consider a user
    efficiency_threshold=0.3  # Efficiency threshold for identifying inefficient users
)

# Display top inefficient PI groups by job count
top_pis = inefficient_pis.head(10)

plt.figure(figsize=(7, 5))
sns.barplot(
    y=top_pis["Account"],
    x=top_pis["Total_GPU_Hours"],
    order=top_pis["Account"].tolist(),  # Only show present values
    orient="h"
)
plt.xlabel("GPU Hours")
plt.ylabel("PI Group")
plt.title("Top 10 Inefficient PI Groups by Weighted VRAM Efficiency Contribution")
plt.show()


In [ ]:
a100_h100_efficiency_analysis = efficiency_analysis.aggregate_metrics_by_gpu()
#convert to a matplotlib table
fig, ax = plt.subplots(figsize=(6, 2 + 0.5 * len(a100_h100_efficiency_analysis)))
ax.axis('off')

# Create the table
table = ax.table(
    cellText=a100_h100_efficiency_analysis.values,
    colLabels=a100_h100_efficiency_analysis.columns,
    rowLabels=a100_h100_efficiency_analysis.index,
    loc='center',
    cellLoc='center'
)


In [ ]:
# Check which columns are present in the jobs DataFrame
efficiency_analysis.additional_metrics()


In [ ]:
efficiency_jobs = efficiency_analysis.calculate_efficiency_metrics()
print("Columns in jobs_df for additional_metrics:")
print(efficiency_jobs.columns)
efficiency_analysis.additional_metrics()

# ...rest of your plotting code...

sns.set(style="whitegrid", font_scale=1.1)

fig, ax = plt.subplots(2, 1, figsize=(18, 14))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

alloc_counts = efficiency_jobs['efficiency_category'].value_counts().sort_index()
sns.barplot(x=alloc_counts.index.astype(str), y=alloc_counts.values, ax=ax[0], palette="Blues_d")
ax[0].set_title('VRAM Allocation Efficiency Distribution')
ax[0].set_xlabel('Allocation Efficiency Category')
ax[0].set_ylabel('Number of Jobs')
ax[0].tick_params(axis='x', rotation=15)

req_counts = efficiency_jobs['duration_category'].value_counts().sort_index()
sns.barplot(x=req_counts.index.astype(str), y=req_counts.values, ax=ax[1], palette="Greens_d")
ax[1].set_title('Request Accuracy Distribution')
ax[1].set_xlabel('Request Accuracy Category')
ax[1].set_ylabel('Number of Jobs')
ax[1].tick_params(axis='x', rotation=15)

plt.show()






In [ ]:
import numpy as np
efficiency_analysis.calculate_efficiency_metrics()
# Only A100 and H100
cat_labels = [
    "Very Low (<10%)", "Low (10-30%)", "Medium (30-60%)", "High (60-100%)"
]
data_counts = {}
data_pcts = {}
gpu_types = (efficiency_analysis.efficiency_df['GPUType']
    .dropna()
    .explode()
    .astype(str)
    .str.strip()
    .str.lower()
    .unique()
)
for gpu in gpu_types:
    jobs = efficiency_analysis.efficiency_df[efficiency_analysis.efficiency_df['GPUType'].apply(
        lambda x, gpu=gpu: gpu in [str(g).strip().lower() for g in (x if isinstance(x, list | np.ndarray)
 else [x])]
    )]
    cat_counts = jobs["efficiency_category"].value_counts().reindex(cat_labels, fill_value=0)
    data_counts[gpu.upper()] = cat_counts.values
    total = cat_counts.sum()
    data_pcts[gpu.upper()] = [f"{(v/total*100):.1f}%" if total else "0.0%" for v in cat_counts.values]
# Build combined table: counts and percentages
table_data = []
for i, _cat in enumerate(cat_labels):
    row = []
    for gpu in gpu_types:
        row.append(f"{data_counts[gpu.upper()][i]}\n({data_pcts[gpu.upper()][i]})")
    table_data.append(row)
# Plot as PNG
fig, ax = plt.subplots(figsize=(8, 3 + 0.7 * len(cat_labels)))
ax.axis('off')
table = ax.table(
    cellText=table_data,
    rowLabels=cat_labels,
    colLabels=[g.upper() for g in gpu_types],
    loc='center',
    cellLoc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(2.0, 2.0)
ax.set_title(f"Efficiency Category Comparison ({80}GB)", fontweight='bold')
save_path = None
if save_path is None:
    save_path = f"eff_cat_{80}GB_comparison.png"
plt.savefig(save_path, bbox_inches='tight', dpi=100)
plt.close()
print(f"Saved efficiency category comparison table to {save_path}")
